# Starting analysis of horror/non-horror collection

There are enough remaining issues with our collection that I'm going to add an extra week on this project. No programming homework this week!

We will start to look at the 2D plots used in the Allison et al. reading.

In [ ]:
import csv, sys, os, re
from collections import Counter
import numpy

from matplotlib import pyplot

word_pattern = re.compile("\w[\w\-\']*\w|\w")

In [ ]:
documents = []

with open("../data/Gutenberg-2019-10-21/metadata.csv", encoding="utf-8") as reader:
    csv_reader = csv.DictReader(reader)
    for document in csv_reader:
        try:
            with open("../data/Gutenberg-2019-10-21/{}".format(document["Filename"]), encoding="utf-8") as reader:
                print(document["Author"] + " / " + document["Title"])

                lines = []
                for line in reader:
                    lines.append(line.rstrip())

                text = " ".join(lines)
                document["Text"] = text
                document["Tokens"] = word_pattern.findall(text)
                
                documents.append(document)
        except Exception as e:
            print("! Problem with {}: {}".format(document["Filename"], e))

In [ ]:
all_counts = Counter()

for document in documents:
    doc_counter = Counter(document["Tokens"])
    all_counts += doc_counter   
    document["TokenCounts"] = doc_counter

In [ ]:
# Construct a fixed vocabulary

vocabulary = [w for w, c in all_counts.most_common()]
vocabulary_size = len(vocabulary)

reverse_vocab = { w: i for i, w in enumerate(vocabulary) }

In [ ]:
def counter_to_vector(counter):
    vector = numpy.zeros(vocabulary_size)
    for word in counter.keys():
        ## look up the integer ID for the string
        word_id = reverse_vocab[word]
        vector[word_id] = counter[word]
    
    return vector

In [ ]:
# Convert counters to vectors
doc_word_matrix = numpy.zeros( (len(documents), len(vocabulary)) )

for doc_id, document in enumerate(documents):
    doc_word_matrix[doc_id,:] = counter_to_vector(document["TokenCounts"])

In [ ]:
doc_word_matrix[:5,:5]

In [ ]:
doc_vectors,S,word_vectors = numpy.linalg.svd(doc_word_matrix, full_matrices=False)

In [ ]:
pyplot.figure(figsize=(14, 8))
pyplot.yticks([])
pyplot.xticks(range(75), vocabulary[:75], rotation="vertical")
pyplot.imshow(word_vectors[:5,:75])
pyplot.show()

In [ ]:
def show_2d(dimension1, dimension2):
    colors = ["red" if "Horror" in doc["Genre"] else "blue" for doc in documents]
    
    pyplot.figure(figsize=(10,10))
    pyplot.scatter(doc_vectors[:,dimension1], doc_vectors[:,dimension2], c=colors)
    for doc_id, document in enumerate(documents):
        pyplot.text(doc_vectors[doc_id,dimension1], 
                   doc_vectors[doc_id,dimension2],
                   document["Title"])
    pyplot.show()

In [ ]:
show_2d(0,1)